In [1]:
import  pandas as pd
import numpy as np

In [2]:
address_clust = pd.read_csv('address_clust.csv')
address_stats = pd.read_csv('address_stats.csv')

In [3]:
address_stats = pd.merge(address_stats, address_clust, on=['address_id'], how="left")
address_stats = address_stats.fillna(0)
address_stats.drop_duplicates(inplace=True)

In [4]:
# Записываем уникальные id всех транзакций 
transactions = address_stats.transaction_id.unique()

Чтобы определить сколько биткоинов кластер 1 отправил в кластер 2, нужно рассмотреть каждую транзакцию отдельно.  
В каждой транзакции берется сумма отправленных биткоинов с кластера 1 и сумма биткоинов, полученных на адреса кластера 2.  
Из двух сумм выбирается минимум, т.к. не всё отправленное с первого кластера может попасть во второй и не всё зачисленное во второй кластер могло быть отправлено с первого.

По аналогии рассчитываются другие ответы.

Для расчета fee необходимо знать со скольки кластеров идет отправка биткоинов в каждой транзакции

In [5]:
# Проверяем из скольки кластеров адресов в каждой транзакции происходит отправка
num = 0
for i in transactions:
    x = len(address_stats[(address_stats.transaction_id == i) & (address_stats.sent > 0)].cluster_id.unique())
    if x > 1:        
        num += 1
        
print(f'Отправок с двух и более кластеров: {num}')

Отправок с двух и более кластеров: 0


Отправка в каждой транзакции происходит только с адресов одного кластера, поэтому рассчет fee будет не сложно реализовать

In [ ]:
%%time
c1_to_c2 = 0
c2_to_c1 = 0
c1_to_c0 = 0
c0_to_c1 = 0
c2_to_c0 = 0
c0_to_c2 = 0
fee1 = 0
fee2 = 0

for i in transactions:    
    
    sent_cluster_0 = address_stats[(address_stats.transaction_id == i) & 
                                   (address_stats.cluster_id == 0)].sent.sum()
    sent_cluster_1 = address_stats[(address_stats.transaction_id == i) & 
                                   (address_stats.cluster_id == 1)].sent.sum()
    sent_cluster_2 = address_stats[(address_stats.transaction_id == i) & 
                                   (address_stats.cluster_id == 2)].sent.sum()
    received_cluster_0 = address_stats[(address_stats.transaction_id == i) & 
                                       (address_stats.cluster_id == 0)].received.sum()
    received_cluster_1 = address_stats[(address_stats.transaction_id == i) & 
                                       (address_stats.cluster_id == 1)].received.sum()
    received_cluster_2 = address_stats[(address_stats.transaction_id == i) & 
                                       (address_stats.cluster_id == 2)].received.sum()

    c1_to_c2 += min(sent_cluster_1, received_cluster_2)
    c2_to_c1 += min(sent_cluster_2, received_cluster_1)
    c1_to_c0 += min(sent_cluster_1, received_cluster_0)
    c0_to_c1 += min(sent_cluster_0, received_cluster_1)
    c2_to_c0 += min(sent_cluster_2, received_cluster_0)
    c0_to_c2 += min(sent_cluster_0, received_cluster_2)
    
    if sent_cluster_1 > 0:
        fee1 += (sent_cluster_1 - address_stats[address_stats.transaction_id == i].received.sum())
        
    if sent_cluster_2 > 0:    
        fee2 += (sent_cluster_2 - address_stats[address_stats.transaction_id == i].received.sum())
    

print(f'Кластер 1 отправил в кластер 2: {c1_to_c2 // 100000000} BTC')
print(f'Кластер 2 отправил в кластер 1: {c2_to_c1 // 100000000} BTC')
print(f'Кластер 1 отправил в кластер 0: {c1_to_c0 // 100000000} BTC')
print(f'Кластер 0 отправил в кластер 1: {c0_to_c1 // 100000000} BTC')
print(f'Кластер 2 отправил в кластер 0: {c2_to_c0 // 100000000} BTC')
print(f'Кластер 0 отправил в кластер 2: {c0_to_c2 // 100000000} BTC')
print(f'Кластер 1 потратил на fee: {fee1} satoshi')
print(f'Кластер 2 потратил на fee: {fee2} satoshi')